# Project Title: Heart Disease Prediction
## Short Description:
- Heart diseases are one of the leading causes of death globally. People with heart disease or who are at high risk of heart disease need early intervention to prevent future undesirable outcomes. In this project, we are going to use a dataset that contains 11 features considered vital in identifying people with heart disease, and we are going to test it with four machine learning models— *KNN, Decision Tree, Random Forest, and Naive Bayes*— that will predict the likelihood of a person having heart disease.

## About the dataset
### Information about the features
1. **Age:** *(years)*
   - patient's age
2. **Sex:** *(M: Male, F: Female)*
   - patient's sex
3. **ChestPainType:** *(TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic)*
   - type of patient's chest pain
4. **RestingBP:** *(mm Hg)*
   - resting blood pressure
5. **Cholesterol:** *(mm/dl)*
   - serum cholesterol
6. **FastingBS:** *(1: if FastingBS > 120 mg/dl, 0: otherwise)*
   - fasting blood sugar
7. **RestingECG:** *[Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]*
   - resting electrocardiogram results
8. **MaxHR:** *(Numeric value between 60 and 202)*
   - maximum heart rate achieved
9.  **ExerciseAngina:** *(Y: Yes, N: No)*
    - exercise-induced angina
10. **Oldpeak:** *(Numeric value measured in depression)*
    - oldpeak = ST 
    - ST depression refers to a finding on an electrocardiogram, wherein the trace in the ST segment is abnormally low below the baseline
11. **ST_Slope:** *(Up: upsloping, Flat: flat, Down: downsloping)*
    - the slope of the peak exercise ST segment
12. **HeartDisease:** *(1: Heart disease, 0: Normal)*
    - target output

---

To learn more about the *source, citation,* and the *creators* of the dataset, click [here](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction).

---